In [1]:
import os
import pandas as pd
import numpy as np
from django.contrib.gis.geos import GEOSGeometry
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
file = '/Users/marcellodebarrosfilho/code/siginsa/gisdata/planilhas/producao_vegetal.xlsx'

In [3]:
projetos = pd.read_excel(file)
projetos = projetos.fillna('')
projetos = projetos.replace({np.nan: None})
projetos.columns = projetos.columns.str.strip()
projetos

,ID Projeto,E-mail institucional do responsável,Título,Palavras Chaves,Início (mês e ano),Término (mês e ano),Status,Objetivo Geral,Público Alvo,Resultados esperados/alcançados,Órgãos de fomento,Instituições parceiras,Documento do projeto - pdf (pesquisas em andamento),Relatório do Projeto - pdf (pesquisas encerradas),Link de artigos ou DOI
0,1,jucilene.araujo@insa.gov.br ou fabiane.costa@i...,Pesquisa e Desenvolvimento Tecnológico em Melh...,Palma forrageira; Melhoramento genético; Doenç...,2020-01-01,2023-01-01,Andamento,Desenvolver pesquisas e desenvolvimento tecnol...,Produtores rurais; Agricultores familiares,Encontar materiais de palma forrageira toleran...,Sudene,"Universidade Federal da Paraíba - UFPB, Campus...",,,
1,2,jucilene.araujo@insa.gov.br,Segurança forrageira e produção madeireira em ...,Palma forrageira; Consórcio; Água residuária,2018-05-01,2022-05-01,Andamento,Estudar a palma forrageira consorciada com esp...,Produtores rurais; Agricultores familiares; Es...,"Aumento da segurança forrageira dos rebanhos, ...",Fundação de Desenvolvimento da Pesquisa - FUNDEP,Centro de Ciências Agrárias da Universidade Fe...,,,https://doi.org/10.1002/agj2.21174;
2,3,jucilene.araujo@insa.gov.br,Reúso de águas residuárias do setor agrícola n...,Palma forrageira; Água residuária; Semiárido,2020-10-01,2023-10-01,Andamento,"Fomentar a implantação de unidades de reúso, v...",Produtores da agricultura familiar,Aumento da segurança alimentar humana e animal...,Ministério do Desenvolvimento Regional - MDR,Universidade Federal Rural de Pernambuco - UFR...,,,


In [4]:
producao_vegetal_list = []

In [5]:
for idx, row in projetos.iterrows():
    id_excel = row['ID Projeto'] 
    email_inst_resp = row['E-mail institucional do responsável']
    titulo = row['Título']
    dt_inicio = row['Início (mês e ano)']
    dt_termino = row['Término (mês e ano)'] if row['Término (mês e ano)'] else None
    objetivo_geral = row['Objetivo Geral']
    resultados = row['Resultados esperados/alcançados']
    documento = row['Documento do projeto - pdf (pesquisas em andamento)']
    relatorio = row['Relatório do Projeto - pdf (pesquisas encerradas)']

    projeto = Projeto.objects.filter(titulo=titulo).first()    
    
    if not projeto:
        projeto = Projeto()
        projeto.id_excel = id_excel
        projeto.email_inst_resp = email_inst_resp
        projeto.titulo = titulo
        projeto.dt_inicio = dt_inicio
        projeto.dt_termino = dt_termino
        projeto.objetivo_geral = objetivo_geral
        projeto.resultados = resultados
        projeto.documento = documento
        projeto.relatorio = relatorio
        projeto.save()
        
        producao_vegetal_list.append(projeto)
        
        link_artigo = row['Link de artigos ou DOI'].split(';')
        for obj in link_artigo:
            obj = obj.strip()
            l = LinkArtigo.objects.filter(link_artigo=obj).first()
            if not l:
                novo_link = LinkArtigo()
                novo_link.link_artigo = obj
                novo_link.projeto = projeto
                novo_link.save()
                
        palavras_chave = row['Palavras Chaves'].split(';')
        for obj in palavras_chave:
            obj = obj.strip().replace('  ', ' ')
            p = PalavraChave.objects.filter(palavra_chave=obj).first()
            if not p:
                nova_palavra = PalavraChave()
                nova_palavra.palavra_chave = obj
                nova_palavra.save()
                projeto.palavras_chave.add(nova_palavra)
            else:
                projeto.palavras_chave.add(p)
                
        publico_alvo = row['Público Alvo'].split(';')
        for obj in publico_alvo:
            obj = obj.strip().replace('  ', ' ')
            p = PublicoAlvo.objects.filter(publico_alvo=obj).first()
            if not p:
                novo_publico = PublicoAlvo()
                novo_publico.publico_alvo = obj
                novo_publico.save()
                projeto.publico_alvo.add(novo_publico)
            else:
                projeto.publico_alvo.add(p)
                
        orgao_fomento = row['Órgãos de fomento'].split(';')
        for obj in orgao_fomento:
            obj = obj.strip().replace('  ', ' ')
            o = OrgaoFomento.objects.filter(orgao_fomento=obj).first()
            if not o:
                novo_orgao = OrgaoFomento()
                novo_orgao.orgao_fomento = obj
                novo_orgao.save()
                projeto.orgao_fomento.add(novo_orgao)
            else:
                projeto.orgao_fomento.add(o)
                
        instituicao_parceira = row['Instituições parceiras'].split(';')
        for obj in instituicao_parceira:
            obj = obj.strip().replace('  ', ' ')
            i = InstituicaoParceira.objects.filter(instituicao_parceira=obj).first()
            if not i:
                nova_inst = InstituicaoParceira()
                nova_inst.instituicao_parceira = obj
                nova_inst.save()
                projeto.instituicao_parceira.add(nova_inst)
            else:
                projeto.instituicao_parceira.add(i)

In [6]:
acoes = pd.read_excel(file, sheet_name='Ações')
acoes = acoes.fillna('')
acoes.columns = acoes.columns.str.strip()
acoes

,ID Projeto,ID Ação,Descrição da ação,UF,Município,Nome da Localidade,Latitude,Longitude,Beneficiários,Quantidade,Fotos,Vídeos
0,1,1,Caracterização morfoagronômica e bromatológica...,PB,Campina Grande,"Estação experimental Prof. Ignácio Salcedo, do...",-7.277126,-35.966466,Produtores rurais; Agricultores familiares,100,,
1,2,1,"Instalação de Campo de Pesquisa/multiplicação,...",PB,Frei Martinho/PB,Unidade de pesquisa Frei martinho/PB,-6.404167,-36.448611,Produtores rurais; Agricultores familiares; Es...,582,,
2,2,2,"Instalação de Campo de Pesquisa/multiplicação,...",RN,São Fernando/RN,Unidade de pesquisa de São Fernando/RN,-6.374722,-37.177778,Produtores rurais; Agricultores familiares; Es...,582,,
3,3,1,Utilizar o esgoto doméstico tratado para subir...,BA,Guanambi/BA,,-14.230533,-42.801749,Produtores rurais; Agricultores familiares; Es...,200,,
4,3,2,Utilizar o esgoto doméstico tratado para subir...,PE,Parnamirim/PE,,-8.087536,-39.578617,Produtores rurais; Agricultores familiares; Es...,200,,
5,3,3,Utilizar o esgoto doméstico tratado para subir...,SE,Nossa Senhora das Dores/SE,,-10.488444,-37.203962,Produtores rurais; Agricultores familiares; Es...,200,,


In [7]:
acoes_producao_vegetal_list = []

In [8]:
for projeto in producao_vegetal_list:
    for idx, row in acoes.iterrows():
        if projeto.id_excel == row['ID Projeto']:
            check_acao = Acao.objects.filter(descricao=row['Descrição da ação']).first()
            if not check_acao:
                acao = Acao()
                acao.projeto = projeto
                acao.descricao = row['Descrição da ação']
                acao.uf = row['UF']
                acao.municipio = row['Município']
                acao.localidade = row['Nome da Localidade']
                acao.quantidade = row['Quantidade']
                geom = GEOSGeometry(f"SRID=4326;POINT({row['Longitude']} {row['Latitude']})")
                if geom.valid:
                    acao.geom = geom
                else:
                    municipio = Municipio.objects.filter(
                        nm_municip__icontains=row['Município'], uf=row['UF']).first()
                    
                    acao.geom = municipio.sede()
                
                acao.save()
                acoes_producao_vegetal_list.append(acao)
                
                beneficiarios = row['Beneficiários'].split(';')
                for obj in beneficiarios:
                    obj = obj.strip()
                    b = Beneficiario.objects.filter(beneficiario=obj).first()
                    if not b:
                        novo_b = Beneficiario()
                        novo_b.beneficiario = obj
                        novo_b.save()
                        acao.beneficiarios.add(novo_b)
                    else:
                        acao.beneficiarios.add(b)

In [9]:
# PublicoAlvo.objects.all().delete()
# PalavraChave.objects.all().delete()
# OrgaoFomento.objects.all().delete()
# InstituicaoParceira.objects.all().delete()
# LinkArtigo.objects.all().delete()
# Beneficiario.objects.all().delete()
# Acao.objects.all().delete()
# Projeto.objects.all().delete()